In [216]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.downloader as api
from nltk.util import ngrams 


import csv
from textstat.textstat import textstat

#TODO combine lists into csv

#toy example
#rows = zip(list1,list2,list3,list4,list5)
#with open(newfilePath, "w") as f:
#    writer = csv.writer(f)
#    for row in rows:
#        writer.writerow(row)

ModuleNotFoundError: No module named 'nlp'

In [136]:
#assorted cleaning
stopWords = stopwords.words('english')

def stem_tokens(tokens, ps):
    stemmed = []
    for item in tokens:
        stemmed.append(ps.stem(item))
    return stemmed
ps = PorterStemmer()

def lemmatize_tokens(tokens, wnl):
    lemmatized = []
    for item in tokens:
        lemmatized.append(wnl.lemmatize(item))
    return lemmatized
wnl = WordNetLemmatizer()


In [119]:

raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [120]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]


{'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
  "The guidelines

In [121]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'truthMedian': 0.33333334,
 'truthMean': 0.2,
 'truthMode': 0.33333334,
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'id': '7'}

In [122]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [123]:
raw_instance_train = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [124]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        non_clickbait_train.append(raw_instance_train[x])

In [125]:
#clickbait_train
clickbait_train[0]

{'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'postMedia': [],
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns about the star goalkeeper.",
  "The

In [126]:
#non_clickbait_train
non_clickbait_train[0]

{'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
  "The guidelines

In [191]:
#Post Text tokenizing and word count manually
postTextTokens = []
postTextTokensNoPun = []
postTextCountTokens = []

for x in range(0, len(raw_instance_train)):
    tokens = word_tokenize(raw_instance_train[x]['postText'][0])
    postTextTokens = [x for x in tokens if x != []]
    postTextTokensNoPun.append([c for c in tokens if c not in punctuation])
    postTextCountTokens.append(postTextTokens)

#print(postTextTokens[0])
#print(len(postTextTokens[0]))
print(postTextTokensNoPun[0])
print(len(postTextTokensNoPun[0]))


['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost']
15


In [213]:
countLongestWord = []

def count_longest_word(word_list):  
    longest_word =  max(word_list , key=len)
    return len(longest_word)

for x in range(0, 1751):
    countLongestWord.append(count_longest_word(postTextTokensNoPun[x]))

print(max(countLongestWord))


26


In [177]:
print(postTextTokensNoPun[915])
count_longest_word(postTextTokensNoPun[915])

['Fury', 'as', 'men', 'throw', 'a', 'cat', 'into', 'a', 'crocodile-infested', 'pond']


18

In [128]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle'][0], removepunct=True))

In [129]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))
targetTitle_coleman_liau_index = []


In [156]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


17581

In [131]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [132]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [217]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

def avg_sim(title, body):
    title = api(title)
    return np.mean(map(lambda x: title.similarity(api(x)), body))


In [218]:
avg_sim(raw_instance_train[0]['targetTitle'], raw_instance_train[0]['targetParagraphs'])

NameError: name 'api' is not defined

In [75]:
text = word_tokenize("And now for something completely different")
nltk.pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

In [76]:
#clickbait_train, emoji
def pos_2gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
for single_data in clickbait_train:
    print(pos_2gram(single_data, "postText"))

[('U.S.', 'NNP'), ('Soccer', 'NNP'), ('should', 'MD'), ('start', 'VB'), ('answering', 'VBG'), ('tough', 'JJ'), ('questions', 'NNS'), ('about', 'IN'), ('Hope', 'NNP'), ('Solo', 'NNP'), (',', ','), ('@', 'NNP'), ('eric_adelson', 'NN'), ('writes', 'NNS'), ('.', '.')]
[('Could', 'MD'), ('light', 'VB'), ('bulbs', 'JJ'), ('hurt', 'VB'), ('your', 'PRP$'), ('health', 'NN'), ('?', '.'), ('One', 'CD'), ('company', 'NN'), ('is', 'VBZ'), ('now', 'RB'), ('putting', 'VBG'), ('warning', 'VBG'), ('labels', 'NNS'), ('on', 'IN'), ('its', 'PRP$'), ('bulbs', 'NN'), (':', ':')]
[('13', 'CD'), ('classic', 'JJ'), ('’', 'NN'), ('00s', 'CD'), ('songs', 'NNS'), ('that', 'WDT'), ('were', 'VBD'), ('actually', 'RB'), ('meant', 'VBN'), ('for', 'IN'), ('other', 'JJ'), ('artists', 'NNS')]
[('Tourists', 'NNS'), ('detained', 'VBD'), ('in', 'IN'), ('Malaysia', 'NNP'), ('for', 'IN'), ('getting', 'VBG'), ('naked', 'VBN'), ('on', 'IN'), ('sacred', 'JJ'), ('mountain', 'NN'), (':', ':')]
[('The', 'DT'), ('brutal', 'JJ'), ('d

[('If', 'IN'), ('modern', 'JJ'), ('art', 'NN'), ('is', 'VBZ'), ('deathly', 'RB'), ('ill', 'JJ'), (',', ','), ('Artenol', 'NNP'), ('may', 'MD'), ('be', 'VB'), ('the', 'DT'), ('cure', 'NN')]
[('The', 'DT'), ('10', 'CD'), ('hardest', 'JJS'), ('Oxbridge', 'NN'), ('degrees', 'NNS'), ('to', 'TO'), ('get', 'VB'), ('accepted', 'VBN'), ('on', 'IN')]
[('Mystery', 'NNP'), ('man', 'NN'), ('donates', 'VBZ'), ('wedding', 'VBG'), ('dress', 'NN'), ('with', 'IN'), ('touching', 'JJ'), ('note', 'NN'), ('about', 'IN'), ('late', 'JJ'), ('wife', 'NN'), ('.', '.')]
[('The', 'DT'), ('U.S.', 'NNP'), ('military', 'JJ'), ('is', 'VBZ'), ('creating', 'VBG'), ('a', 'DT'), ('robot', 'NN'), ('revolution', 'NN'), ('by', 'IN'), ('@', 'NNP'), ('pogue', 'NN')]
[('This', 'DT'), ('Tumblr', 'NNP'), ('account', 'NN'), ('will', 'MD'), ('nail', 'VB'), ('your', 'PRP$'), ('personality', 'NN'), ('down', 'RP'), ('in', 'IN'), ('a', 'DT'), ('second', 'JJ')]
[('What', 'WP'), ("'s", 'VBZ'), ('really', 'RB'), ('in', 'IN'), ('that', 'DT

[('DOJ', 'NN'), (':', ':'), ('17-year-old', 'JJ'), ('Virginia', 'NNP'), ('teen', 'NN'), ('accused', 'VBN'), ('of', 'IN'), ('using', 'VBG'), ('social', 'JJ'), ('media', 'NNS'), ('to', 'TO'), ('recruit', 'VB'), ('and', 'CC'), ('raise', 'VB'), ('money', 'NN'), ('for', 'IN'), ('ISIS', 'NNP'), (':', ':'), ('-', ':'), ('@', 'NN'), ('MLevineReports', 'NNS')]
[('Could', 'MD'), ('your', 'PRP$'), ('imagination', 'NN'), ('be', 'VB'), ('making', 'VBG'), ('you', 'PRP'), ('FAT', 'NNP'), ('?', '.')]
[('Why', 'WRB'), ('one', 'CD'), ('failed', 'VBD'), ('asylum', 'NN'), ('seeker', 'NN'), ('trended', 'VBD'), ('-', ':'), ('while', 'IN'), ('many', 'JJ'), ('others', 'NNS'), ('do', 'VBP'), ('not', 'RB')]
[('#', '#'), ('JurassicWorld', 'NNP'), ('a', 'DT'), ('worthy', 'JJ'), ('successor', 'NN'), ('to', 'TO'), ("'Jurassic", 'VB'), ('Park', 'NNP'), ("'", 'POS'), ('?', '.')]
[('You', 'PRP'), ("'re", 'VBP'), ('essentially', 'RB'), ('saying', 'VBG'), ('``', '``'), ('tea', 'NN'), ('tea', 'NN'), ("''", "''")]
[('This

[('RT', 'NNP'), ('@', 'NNP'), ('BuzzFeedBooks', 'NNP'), (':', ':'), ('53', 'CD'), ('of', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('opening', 'NN'), ('lines', 'NNS'), ('in', 'IN'), ('literature', 'NN'), (':', ':')]
[('Behind', 'IN'), ('technologies', 'NNS'), ('that', 'WDT'), ('empower', 'VBP'), ('people', 'NNS'), ('and', 'CC'), ('the', 'DT'), ('passions', 'NNS'), ('they', 'PRP'), ('ignite', 'VBP')]
[('Executive', 'JJ'), ('pay', 'NN'), ('is', 'VBZ'), ('getting', 'VBG'), ('a', 'DT'), ('closer', 'JJR'), ('look', 'NN'), ('from', 'IN'), ('activist', 'JJ'), ('investors', 'NNS'), (':', ':')]
[('We', 'PRP'), ('went', 'VBD'), ('to', 'TO'), ('see', 'VB'), ('``', '``'), ('Jurassic', 'JJ'), ('World', 'NNP'), ("''", "''"), ('with', 'IN'), ('2', 'CD'), ('paleontologists', 'NNS')]
[('#', '#'), ('ICYMI', 'NNP'), (':', ':'), ('Hack', 'VBP'), ('the', 'DT'), ('vote', 'NN'), (':', ':'), ('Cyber', 'NNP'), ('experts', 'NNS'), ('say', 'VBP'), ('ballot', 'NN'), ('machines', 'NNS'), ('easy', 'JJ'), ('targets', '

[('What', 'WP'), ('is', 'VBZ'), ('Sarin', 'NNP'), ('?', '.'), ('In', 'IN'), ('Syria', 'NNP'), (',', ','), ('potent', 'NN'), ('gas', 'NN'), ('believed', 'VBN'), ('to', 'TO'), ('be', 'VB'), ('behind', 'IN'), ('fatal', 'JJ'), ('attack', 'NN'), ('#', '#'), ('FOXNewsWorld', 'NNP')]
[('Pepsi', 'NNP'), ('has', 'VBZ'), ('pulled', 'VBN'), ('its', 'PRP$'), ('Kendall', 'NNP'), ('Jenner', 'NNP'), ('protest', 'JJS'), ('ad', 'NN'), ('after', 'IN'), ('a', 'DT'), ('day', 'NN'), ('of', 'IN'), ('intense', 'JJ'), ('criticism', 'NN')]
[('Trump', 'NNP'), ('aide', 'NN'), ('Steve', 'NNP'), ('Bannon', 'NNP'), ('removed', 'VBD'), ('from', 'IN'), ('National', 'NNP'), ('Security', 'NNP'), ('Council', 'NNP')]
[('Leonard', 'NNP'), ('Fournette', 'NNP'), ('is', 'VBZ'), ('noticeably', 'RB'), ('slimmer', 'JJR'), ('than', 'IN'), ('he', 'PRP'), ('was', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('NFL', 'NNP'), ('scouting', 'NN'), ('combine', 'NN'), ('.', '.')]
[('The', 'DT'), ('price', 'NN'), ('of', 'IN'), ('an', 'DT'), ('AK-

[('Rory', 'NNP'), ('Feek', 'NNP'), ('remembers', 'NNS'), ('late', 'JJ'), ('wife', 'NN'), ('Joey', 'NNP'), (',', ','), ('a', 'DT'), ('year', 'NN'), ('after', 'IN'), ('her', 'PRP$'), ('death', 'NN')]
[('``', '``'), ('This', 'DT'), ('behaviour', 'NN'), ('hurts', 'VBZ'), ('fellow', 'JJ'), ('marines', 'NNS'), (',', ','), ('family', 'NN'), ('members', 'NNS'), (',', ','), ('and', 'CC'), ('civilians', 'NNS'), (',', ','), ("''", "''"), ('said', 'VBD'), ('Sgt', 'NNP'), ('Maj', 'NNP'), ('Ronald', 'NNP'), ('Green', 'NNP'), ('in', 'IN'), ('a', 'DT'), ('statement', 'NN')]
[('There', 'EX'), ('’', 'JJ'), ('s', 'NN'), ('already', 'RB'), ('a', 'DT'), ('rumor', 'NN'), ('about', 'IN'), ('the', 'DT'), ('2019', 'CD'), ('iPhones', 'NNS'), ('claiming', 'VBG'), ('they', 'PRP'), ("'ll", 'MD'), ('all', 'DT'), ('have', 'VBP'), ('OLED', 'VBN'), ('screens', 'NNS')]
[('This', 'DT'), ('one', 'CD'), ('chart', 'NN'), ('shows', 'VBZ'), ('what', 'WP'), ('Goldman', 'NNP'), ('expects', 'VBZ'), ('Donald', 'NNP'), ('Trump', 

[('THE', 'DT'), ('PORK', 'NNP'), ('MUST', 'NNP'), ('FLOW', 'NNP'), ('.', '.')]
[('How', 'WRB'), ('ancestral', 'JJ'), ('climates', 'NNS'), ('may', 'MD'), ('have', 'VB'), ('shaped', 'VBN'), ('your', 'PRP$'), ('nose', 'NN')]
[('.', '.'), ('@', 'CC'), ('rickyrozay', 'JJ'), ('drops', 'NNS'), ('his', 'PRP$'), ("'Rather", 'NN'), ('You', 'PRP'), ('Than', 'IN'), ('Me', 'NNP'), ("'", 'POS'), ('album', 'NN'), (',', ','), ('f/', 'JJ'), ('Future', 'NNP'), (',', ','), ('Gucci', 'NNP'), ('Mane', 'NNP'), (',', ','), ('Nas', 'NNP'), (',', ','), ('and', 'CC'), ('more', 'JJR'), ('.', '.')]
[('Amber', 'NNP'), ('Harrison', 'NNP'), ('hires', 'VBZ'), ('high-profile', 'JJ'), ('#', '#'), ('barrister', 'NN'), ('in', 'IN'), ('battle', 'NN'), ('against', 'IN'), ('Seven', 'NNP'), ('West', 'NNP'), ('Media', 'NNP')]
[('Leigh', 'NNP'), ('heap', 'JJ'), ('pressure', 'NN'), ('on', 'IN'), ('Warrington', 'NNP'), ('after', 'IN'), ('Gareth', 'NNP'), ('Hock', 'NNP'), ('leads', 'VBZ'), ('the', 'DT'), ('way', 'NN'), ('via', 'I

[('oh', 'UH'), ('?', '.')]
[('Lavish', 'JJ'), ('Lifestyle', 'NNP'), (':', ':'), ('North', 'JJ'), ('Korean', 'JJ'), ('dictator', 'NN'), ("'s", 'POS'), ('resort', 'NN'), ('revealed', 'VBD'), ('as', 'IN'), ('his', 'PRP$'), ('people', 'NNS'), ('starve', 'VBP')]
[('Animated', 'NNP'), ('map', 'NN'), ('of', 'IN'), ('what', 'WP'), ('Earth', 'NNP'), ('would', 'MD'), ('look', 'VB'), ('like', 'IN'), ('if', 'IN'), ('all', 'PDT'), ('the', 'DT'), ('ice', 'NN'), ('melted', 'VBD'), ('#', '#'), ('EarthDay', 'NNP')]
[('Pakistani', 'NNP'), ('man', 'NN'), (',', ','), ('50', 'CD'), (',', ','), ('is', 'VBZ'), ('addicted', 'VBN'), ('to', 'TO'), ('eating', 'VBG'), ('TREES', 'NNP'), ('after', 'IN'), ('eating', 'VBG'), ('leaves', 'NNS'), ('when', 'WRB'), ('he', 'PRP'), ('was', 'VBD'), ('poor', 'JJ')]
[('Phil', 'NNP'), ('Simms', 'NNP'), ('shares', 'NNS'), ('his', 'PRP$'), ('perspective', 'NN'), ('on', 'IN'), ('CBS', 'NNP'), ('deciding', 'VBG'), ('to', 'TO'), ('replace', 'VB'), ('him', 'PRP'), ('with', 'IN'), ('T

[('Marine', 'NNP'), ('veteran', 'NN'), ('who', 'WP'), ('lost', 'VBD'), ('both', 'DT'), ('legs', 'NNS'), ('in', 'IN'), ('Afghanistan', 'NNP'), ('sworn', 'NN'), ('in', 'IN'), ('as', 'IN'), ('NY', 'NNP'), ('police', 'NNS'), ('officer', 'NN')]
[('Joe', 'NNP'), ('Biden', 'NNP'), ('regrets', 'VBZ'), ('not', 'RB'), ('becoming', 'VBG'), ('president', 'NN')]
[('Zayn', 'NNP'), ('Malik', 'NNP'), ("'s", 'POS'), ('new', 'JJ'), ('song', 'NN'), ('#', '#'), ('StillGotTime', 'NNP'), ('feat', 'NN'), ('.', '.'), ('PartyNextDoor', 'NNP'), ('has', 'VBZ'), ('arrived', 'VBN')]
[('Watch', 'VB'), ('the', 'DT'), ('video', 'NN'), ('for', 'IN'), ('Iggy', 'NNP'), ('Azalea', 'NNP'), ("'s", 'POS'), ('new', 'JJ'), ('single', 'JJ'), ('``', '``'), ('Mo', 'NNP'), ('Bounce', 'NNP'), ('.', '.'), ("''", "''")]
[('Trump', 'NNP'), ("'s", 'POS'), ('links', 'NNS'), ('to', 'TO'), ('a', 'DT'), ('convicted', 'VBN'), ('felon', 'NN'), ('who', 'WP'), ('stabbed', 'VBD'), ('a', 'DT'), ('man', 'NN'), ('with', 'IN'), ('a', 'DT'), ('marg

[('The', 'DT'), ('movie', 'NN'), ('musical', 'JJ'), ('looked', 'VBD'), ('to', 'TO'), ('be', 'VB'), ('dead', 'JJ'), ('and', 'CC'), ('buried', 'VBD'), ('only', 'RB'), ('a', 'DT'), ('year', 'NN'), ('ago', 'RB'), ('.', '.'), ('Then', 'RB'), ('came', 'VBD'), ('``', '``'), ('La', 'NNP'), ('La', 'NNP'), ('Land', 'NNP'), ("''", "''")]
[('The', 'DT'), ('#', '#'), ('TPP', 'NNP'), ('is', 'VBZ'), ('``', '``'), ('dead', 'JJ'), ('in', 'IN'), ('the', 'DT'), ('water', 'NN'), ("''", "''"), ('and', 'CC'), ('pursuing', 'VBG'), ('it', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('``', '``'), ('waste', 'NN'), ('of', 'IN'), ('time', 'NN'), ("''", "''"), (',', ','), ('says', 'VBZ'), ('@', 'NNP'), ('billshortenmp', 'NN'), ('#', '#'), ('auspol', 'JJ'), ('#', '#'), ('ausbiz', 'NN')]
[('MORE', 'RBR'), (':', ':'), ('Packers', 'NNS'), ('upset', 'VBP'), ('top-seeded', 'JJ'), ('Cowboys', 'NNP'), ('in', 'IN'), ('NFL', 'NNP'), ('playoff', 'NN'), ('thriller', 'NN')]
[('Employees', 'NNS'), ('of', 'IN'), ('@', 'NN'), ('WeChatApp

In [78]:
pos_2gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [79]:
#calculate tags in a data
def cal_pos_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [81]:
cal_pos_tag(clickbait_train[0], "postText", "NNP")

tags:  ('U.S.', 'NNP')
tags:  ('Soccer', 'NNP')
tags:  ('should', 'MD')
tags:  ('start', 'VB')
tags:  ('answering', 'VBG')
tags:  ('tough', 'JJ')
tags:  ('questions', 'NNS')
tags:  ('about', 'IN')
tags:  ('Hope', 'NNP')
tags:  ('Solo', 'NNP')
tags:  (',', ',')
tags:  ('@', 'NNP')
tags:  ('eric_adelson', 'NN')
tags:  ('writes', 'NNS')
tags:  ('.', '.')


5